<a href="https://colab.research.google.com/github/sorielkobayashi/Custom-Spleeter-Colab/blob/main/Neural%20Audio%20Stem%20Separation%20(Espa%C3%B1ol).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Spleeter Colab

Esta es una extensión para [Deezer Spleeter](https://github.com/deezer/spleeter) que se ejecuta en Google Colab, utilizando Google Drive como almacenamiento de datos.

El valor de `input` puede ser
1.   Ruta a un archivo de audio individual ubicado en su Google Drive.
2.   Ruta a un directorio en su Google Drive que contenga varios archivos de audio. Cada archivo de audio encontrado en el directorio será procesado individualmente.
3.   URL individual de YouTube
4.   Múltiples URLs de YouTube separadas por espacios. Cada vídeo de YouTube será procesado individualmente.

La entrada será separada en 4 stems: voz, batería, bajo y otros. Los stems se guardarán como archivos WAV en su Google Drive. Introduzca `output_dir` relativo a su raíz de Google Drive.

In [1]:
#@title Setup
#@markdown This cell needs to be run only once. It will mount your Google Drive and setup prerequisites.

import os
from google.colab import drive, output
drive.mount('/content/drive')
drive_root = '/content/drive/MyDrive/'

# inhagcutils
!pip -q install import-ipynb
!curl -s -O https://raw.githubusercontent.com/olaviinha/inhagcutils/master/inhagcutils.ipynb
import import_ipynb
from inhagcutils import *
!gsutil -q -m cp -R gs://neural-research/olaviinha/spleeter-configs/custom-4stems-22kHz-z.json /content/cfg.json

#@title Install
!apt -q install ffmpeg
!pip -q install spleeter

output.clear()
print('Using config gs://neural-research/olaviinha/spleeter-configs/custom-4stems-22kHz-z.json')
op(c.ok, 'Setup finished.')


Using config gs://neural-research/olaviinha/spleeter-configs/custom-4stems-22kHz-z.json
Setup finished.


In [2]:
#@title Separate

input = "/content/04 - She Loves You (2009 Digital Remaster).flac" #@param {type:"string"}
output_dir = "/stems2" #@param {type:"string"}

#----------------------------------------------------------------------------#

import os
from os.path import isfile, join
from glob import glob

input_type = check_input_type(input)
tmp_dir = '/content/tmp/'
seps_dir = fix_path(drive_root+output_dir)
#seps_dir = '/content/tmp/seps'

if not os.path.isdir(tmp_dir):
  create_dirs([tmp_dir])
# if not os.path.isdir(output_dir):
#   create_dirs([seps_dir])

if output_dir == '' or output_dir == 'same':
  output_dir = fix_path(drive_root+path_dir(input))
else:
  output_dir = fix_path(drive_root+output_dir)
  if not os.path.isdir(output_dir):
    create_dirs([output_dir])

if input_type == 'unknown':
  input = drive_root+input
  input_type = check_input_type(input)
  if input_type=='dir':
    input = fix_path(input)

if input_type == 'youtube':
  !pip -q install youtube-dl
  !youtube-dl --restrict-filenames -x --no-continue --audio-format wav -o "{tmp_dir}%(title)s.%(ext)s" {input}

from spleeter.separator import Separator
separator = Separator('/content/cfg.json')

if input_type == "dir" or input_type == "youtube":
  if input_type == "dir":
    target = input
  if input_type == "youtube":
    target = dir_tmp
  file_list = list_audio(target)
  for audiofile in file_list:
    op(c.title, '\nSeparate '+audiofile+' to '+seps_dir.replace(drive_root, '')+basename(input))
    # Spleeter Python API is outdated for Colab
    # separator.separate_to_file(audiofile, seps_dir)
    !spleeter separate -p /content/cfg.json -o "{output_dir}" "{audiofile}"
    op(c.ok, 'FIN.', 'Files saved under directory '+output_dir.replace(drive_root, '')+basename(audiofile)+'/')
elif input_type == "file":
  # Spleeter Python API is outdated for Colab
  # separator.separate_to_file(input, seps_dir)
  !spleeter separate -p /content/cfg.json -o "{seps_dir}" "{input}"
  op(c.ok, 'FIN.', 'Files saved under directory '+output_dir.replace(drive_root, '')+basename(input)+'/')
else:
  print("We're not sure what you want to separate. Please check the input field.")

/usr/local/lib/python3.7/dist-packages/resampy/interpn.py:114: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  _resample_loop_p(x, t_out, interp_win, interp_delta, num_table, scale, y)
INFO:spleeter:Downloading model archive https://github.com/deezer/spleeter/releases/download/v1.4.0/4stems.tar.gz
INFO:spleeter:Validating archive checksum
INFO:spleeter:Extracting downloaded 4stems archive
INFO:spleeter:4stems model file(s) extracted
tcmalloc: large alloc 1258348544 bytes == 0xb183a000 @  0x7fa58320d1e7 0x7fa54c8bff42 0x7fa54f2c4c5a 0x7fa54f6655c4 0x7fa54f6b5b1f 0x7fa54f6b7769 0x7fa54f6b8757 0x7fa54f6b9e6e 0x7fa54f6be942 0x7fa54f6bee3f 0x7fa54555ee54 0x7fa5455531be 0x7fa54aa09021 0x7fa54aa05d33 0x7fa545c9b8b5 0x7fa582bd16db 0x7fa582f0a61f
tcmalloc: large alloc 1258348544 bytes == 0xfc848000 @  0x7fa58320d1e7 0x7fa54c8bff42 0x7fa54f2c4c5a 0x7fa54f665

In [3]:
#@title Separate

input = "/content/07 - Michelle (2009 Digital Remaster).flac" #@param {type:"string"}
output_dir = "/stems2" #@param {type:"string"}

#----------------------------------------------------------------------------#

import os
from os.path import isfile, join
from glob import glob

input_type = check_input_type(input)
tmp_dir = '/content/tmp/'
seps_dir = fix_path(drive_root+output_dir)
#seps_dir = '/content/tmp/seps'

if not os.path.isdir(tmp_dir):
  create_dirs([tmp_dir])
# if not os.path.isdir(output_dir):
#   create_dirs([seps_dir])

if output_dir == '' or output_dir == 'same':
  output_dir = fix_path(drive_root+path_dir(input))
else:
  output_dir = fix_path(drive_root+output_dir)
  if not os.path.isdir(output_dir):
    create_dirs([output_dir])

if input_type == 'unknown':
  input = drive_root+input
  input_type = check_input_type(input)
  if input_type=='dir':
    input = fix_path(input)

if input_type == 'youtube':
  !pip -q install youtube-dl
  !youtube-dl --restrict-filenames -x --no-continue --audio-format wav -o "{tmp_dir}%(title)s.%(ext)s" {input}

from spleeter.separator import Separator
separator = Separator('/content/cfg.json')

if input_type == "dir" or input_type == "youtube":
  if input_type == "dir":
    target = input
  if input_type == "youtube":
    target = dir_tmp
  file_list = list_audio(target)
  for audiofile in file_list:
    op(c.title, '\nSeparate '+audiofile+' to '+seps_dir.replace(drive_root, '')+basename(input))
    # Spleeter Python API is outdated for Colab
    # separator.separate_to_file(audiofile, seps_dir)
    !spleeter separate -p /content/cfg.json -o "{output_dir}" "{audiofile}"
    op(c.ok, 'FIN.', 'Files saved under directory '+output_dir.replace(drive_root, '')+basename(audiofile)+'/')
elif input_type == "file":
  # Spleeter Python API is outdated for Colab
  # separator.separate_to_file(input, seps_dir)
  !spleeter separate -p /content/cfg.json -o "{seps_dir}" "{input}"
  op(c.ok, 'FIN.', 'Files saved under directory '+output_dir.replace(drive_root, '')+basename(input)+'/')
else:
  print("We're not sure what you want to separate. Please check the input field.")

/usr/local/lib/python3.7/dist-packages/resampy/interpn.py:114: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  _resample_loop_p(x, t_out, interp_win, interp_delta, num_table, scale, y)
tcmalloc: large alloc 1468088320 bytes == 0xccba2000 @  0x7f875d9871e7 0x7f8727039f42 0x7f8729a3ec5a 0x7f8729ddf5c4 0x7f8729e2fb1f 0x7f8729e31769 0x7f8729e32757 0x7f8729e33e6e 0x7f8729e38942 0x7f8729e38e3f 0x7f871fcd8e54 0x7f871fccd1be 0x7f8725183021 0x7f872517fd33 0x7f87204158b5 0x7f875d34b6db 0x7f875d68461f
tcmalloc: large alloc 1468088320 bytes == 0x1243b6000 @  0x7f875d9871e7 0x7f8727039f42 0x7f8729a3ec5a 0x7f8729ddf5c4 0x7f8729e2fb1f 0x7f8729e31769 0x7f8729e32757 0x7f8729e33e6e 0x7f8729e38942 0x7f8729e38e3f 0x7f871fcd8e54 0x7f871fccd1be 0x7f8725183021 0x7f872517fd33 0x7f87204158b5 0x7f875d34b6db 0x7f875d68461f
tcmalloc: large alloc 1468088320 bytes == 0x12275400

In [ ]:
#@title Comprimir Carpeta

input = "/content/drive/MyDrive/stems2/04 - She Loves You (2009 Digital Remaster)" #@param {type:"string"}


#----------------------------------------------------------------------------#



from google.colab import files
!apt-get install rar
!rar -?
!rar a "{input}".rar "{input}"



In [18]:
#@title Comprimir Carpeta

input = "/content/drive/MyDrive/stems2/04 - She Loves You (2009 Digital Remaster)" #@param {type:"string"}


#----------------------------------------------------------------------------#



from google.colab import files
!apt-get install rar
!rar  a -m5 "{input}".rar "{input}"



Reading package lists... Done
Building dependency tree       
Reading state information... Done
rar is already the newest version (2:5.5.0-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.

RAR 5.50   Copyright (c) 1993-2017 Alexander Roshal   11 Aug 2017
Trial version             Type 'rar -?' for help

Evaluation copy. Please register.

Creating archive /content/drive/MyDrive/stems2/04 - She Loves You (2009 Digital Remaster).rar

Adding    /content/drive/MyDrive/stems2/04 - She Loves You (2009 Digital Remaster)/vocals.wav       4%  8% 12% 16% 21% 24%  OK 
Adding    /content/drive/MyDrive/stems2/04 - She Loves You (2009 Digital Remaster)/drums.wav      29% 33% 37% 41% 46% 49%  OK 
Adding    /content/drive/MyDrive/stems2/04 - She Loves You (2009 Digital Remaster)/bass.wav     